In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Database connection parameters
DB_USER = 'root'
DB_PASSWORD = '*****'
DB_HOST = 'localhost'  # or your database host
DB_PORT = '****'       # or your database port
DB_NAME = '*****'

# Function to create a database connection
def create_connection():
    connection = None
    try:
        connection = mysql.connector.connect(
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

# Load the Excel file
excel_file_path = r"file_path"  # Update with your Excel file path
df = pd.read_excel(excel_file_path)

# Function to get farmer_id and farm_id based on last_name, first_name, and farm_number
def get_ids(connection, row):
    # Query to get farmer_id
    farmer_query = f"""
    SELECT farmer_id FROM farmer 
    WHERE last_name = '{row['last_name']}' AND first_name = '{row['first_name']}';
    """
    cursor = connection.cursor()
    cursor.execute(farmer_query)
    farmer_id = cursor.fetchone()
    
    # Ensure all results are fetched before creating a new cursor
    cursor.fetchall()  # Fetch all results to avoid unread result error
    
    # Get the farmer_id
    farmer_id = farmer_id[0] if farmer_id else None

    # Query to get farm_id based on farm_number and farmer_id
    farm_number = row['farm_number']
    farm_query = """
    SELECT farm_id FROM farm 
    WHERE farm_number = %s AND farmer_id = %s;
    """
    cursor.execute(farm_query, (farm_number, farmer_id))
    farm_id = cursor.fetchone()
    
    # Ensure all results are fetched before closing the cursor
    cursor.fetchall()  # Fetch all results to avoid unread result error
    
    cursor.close()
    
    return farmer_id, farm_id[0] if farm_id else None

# Prepare a list to hold the new tree data records
new_tree_data = []

# Create a database connection
connection = create_connection()

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    try:
        # Get farmer_id and farm_id
        farmer_id, farm_id = get_ids(connection, row)

        if farmer_id is not None and farm_id is not None:
            # Create a new record for tree_data, ignoring fruit_set_percentage
            new_record = {
                'farm_id': farm_id,
                'farmer_id': farmer_id,
                'tree_number': row['tree_number'] if pd.notnull(row['tree_number']) else None,
                'tree_geopoint': row['tree_geopoint'] if pd.notnull(row['tree_geopoint']) else None,  # Ensure this is in the correct format
                'picture_1_url': row['picture_1_url'] if pd.notnull(row['picture_1_url']) else None,
                'picture_2_url': row['picture_2_url'] if pd.notnull(row['picture_2_url']) else None,
                'picture_3_url': row['picture_3_url'] if pd.notnull(row['picture_3_url']) else None,
                'picture_4_url': row['picture_4_url'] if pd.notnull(row['picture_4_url']) else None,
                'picture_5_url': row['picture_5_url'] if pd.notnull(row['picture_5_url']) else None,
                'picture_6_url': row['picture_6_url'] if pd.notnull(row['picture_6_url']) else None,
                'picture_7_url': row['picture_7_url'] if pd.notnull(row['picture_7_url']) else None,
                'picture_8_url': row['picture_8_url'] if pd.notnull(row['picture_8_url']) else None,
            }
            new_tree_data.append(new_record)
        else:
            print(f"Farmer or Farm not found for row {index}. Skipping...")
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Insert the new records into the tree_data table
insert_query = """
INSERT INTO tree_data (farm_id, farmer_id, tree_number, tree_geopoint, 
                       picture_1_url, picture_2_url, picture_3_url, 
                       picture_4_url, picture_5_url, picture_6_url, 
                       picture_7_url, picture_8_url)
VALUES (%s, %s, %s, ST_GeomFromText(%s), %s, %s, %s, %s, %s, %s, %s, %s);
"""

cursor = connection.cursor()

for record in new_tree_data:
    try:
        cursor.execute(insert_query, (
            record['farm_id'],
            record['farmer_id'],
            record['tree_number'],
            record['tree_geopoint'],  # Ensure this is in the correct format for POINT
            record['picture_1_url'],
            record['picture_2_url'],
            record['picture_3_url'],
            record['picture_4_url'],
            record['picture_5_url'],
            record['picture_6_url'],
            record['picture_7_url'],
            record['picture_8_url'],
        ))
    except Exception as e:
        print(f"Error inserting record {record}: {e}")

# Commit the transaction
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("Data uploaded successfully!")